In [1]:
import tweepy
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
df_credentials = pd.read_csv("credentials.txt", sep = ",") #all your twitter and gmail credentials goes in here

<h1>Access twitter API and create an api Instance </h1>

In [3]:
auth = tweepy.OAuthHandler(df_credentials.value[0], df_credentials.value[1])
auth.set_access_token(df_credentials.value[2],df_credentials.value[3])
api = tweepy.API(auth)

In [4]:
#Get twitter trends object
tweet_trends = api.trends_available()

In [5]:
#function that gets the list of twitter trends
def getTweetTrends(ls):
    dict_countries = {}
    data = []
    
    for trend in tweet_trends:
        if trend['name'] in ls:
            dict_countries[trend['name']] = trend['woeid']
            
    for country in dict_countries.keys():
        trends_place = api.trends_place(id = dict_countries[country])
        
        for terms in trends_place[0]['trends']:
            dict_ = {}
            #print(terms)
            dict_['country'] = country
            dict_['name'] = terms['name']
            dict_['tweet_volume'] = terms['tweet_volume']
            data.append(dict_)
            #print(data)
    
    df = pd.DataFrame(data)
    return df   

In [6]:
#pass the list of countries you would like to see the trends for and add it to the list
countries_list = ['United States', 'India']
df = getTweetTrends(countries_list)

In [7]:
df.sort_values(by = "tweet_volume", ascending = False, inplace=True)

<h1> Send twitter trends as a email using your gmail account </h1>

In [8]:
#for reference use this link: http://stackabuse.com/how-to-send-emails-with-gmail-using-python/
try:  
    server_ssl = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server_ssl.ehlo()   # optional
except:  
    print 'Something went wrong...'

In [9]:
gmail_user = df_credentials.value[4]  #you@gmail.com 
gmail_password = df_credentials.value[5] #gmail app password

sent_from = gmail_user  
to = [df_credentials.value[4]] #Email To

msg = MIMEMultipart('alternative')
msg['Subject'] = 'Twitter Trends'

part = MIMEText(df.to_html().encode('utf-8'), 'html')

msg.attach(part)

try:  
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(sent_from, to, msg.as_string())
    server.close()

    print 'Email sent!'
except:  
    print 'Something went wrong...'

Email sent!
